In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import random
import pathlib
import cv2
import seaborn as sns

import os
os.getcwd()

In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import random
from skimage import exposure

# Directory containing MRI images
data_dir = 'D:\\Brain Cancer\\Dataset\\Brain_Cancer raw MRI data\\Brain_Cancer'
save_dir = 'D:\\Brain Cancer\\Dataset\\Brain_Cancer_processed_2'  # Directory to save preprocessed images

# Create directory to save preprocessed images
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Function to apply intensity normalization (Z-score normalization)
def normalize_image(image):
    mean = np.mean(image)
    std = np.std(image)
    normalized_image = (image - mean) / std
    return normalized_image

# Function for data augmentation
def augment_image(image):
    aug_images = []

    # Flipping horizontally
    flipped_image = cv2.flip(image, 1)
    aug_images.append(flipped_image)

    # Adding noise
    noise = np.random.normal(0, 0.0023**0.5, image.shape)
    noisy_image = image + noise
    aug_images.append(np.clip(noisy_image, 0, 1))  # Ensure pixel values are in [0, 1]

    # Adjusting contrast and brightness using histogram equalization
    contrast_image = exposure.equalize_hist(image)  # Histogram equalization to enhance contrast
    aug_images.append(contrast_image)

    # Rotations
    rows, cols = image.shape[:2]
    for angle in [-13, -9, +9, +13]:
        M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
        rotated_image = cv2.warpAffine(image, M, (cols, rows))
        aug_images.append(rotated_image)

    # Apply random affine transformation
    for _ in range(2):  # Generate two affine transformations
        pts1 = np.float32([[50, 50], [200, 50], [50, 200]])
        pts2 = pts1 + np.random.uniform(-10, 10, size=pts1.shape).astype(np.float32)
        M_affine = cv2.getAffineTransform(pts1, pts2)
        affine_image = cv2.warpAffine(image, M_affine, (cols, rows))
        aug_images.append(affine_image)

    return aug_images

# Loop through all subfolders and images
for subfolder in os.listdir(data_dir):
    subfolder_path = os.path.join(data_dir, subfolder)
    
    # Ensure the path is a directory
    if os.path.isdir(subfolder_path):
        # Create corresponding subfolder in the save directory
        save_subfolder = os.path.join(save_dir, subfolder)
        if not os.path.exists(save_subfolder):
            os.makedirs(save_subfolder)
        
        # Loop through all images in the current subfolder
        for img_name in tqdm(os.listdir(subfolder_path)):
            # Read the image
            img_path = os.path.join(subfolder_path, img_name)
            image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Load as grayscale
            if image is None:
                continue

            # Step 1: Resize to 256x256
            resized_image = cv2.resize(image, (256, 256))

            # Step 2: Normalize (Z-score normalization)
            normalized_image = normalize_image(resized_image)

            # Step 3: Apply augmentation
            augmented_images = augment_image(normalized_image)

            # Save the processed images
            base_name, ext = os.path.splitext(img_name)
            cv2.imwrite(os.path.join(save_subfolder, f"{base_name}_resized{ext}"), (normalized_image * 255).astype(np.uint8))

            # Save augmented images
            for i, aug_image in enumerate(augmented_images):
                cv2.imwrite(os.path.join(save_subfolder, f"{base_name}_aug_{i}{ext}"), (aug_image * 255).astype(np.uint8))

print(f"Preprocessing complete. Processed images saved to {save_dir}.")


In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import random

# Directory containing MRI images
data_dir = 'D:\\Brain Cancer\\Dataset\\Brain_Cancer raw MRI data\\Brain_Cancer'
save_dir = 'D:\\Brain Cancer\\Dataset\\Brain_Cancer_processed3'  # Directory to save preprocessed images

# Create directory to save preprocessed images
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Function to apply Z-score normalization (mean subtraction and standard deviation division)
def normalize_image(image):
    mean = np.mean(image)
    std = np.std(image)
    normalized_image = (image - mean) / (std + 1e-7)  # Avoid division by zero
    return normalized_image

# Function for more conservative data augmentation
def augment_image(image):
    aug_images = []

    # Flip horizontally (minimal transformation)
    flipped_image = cv2.flip(image, 1)
    aug_images.append(flipped_image)

    # Adding slight noise (milder transformation)
    noise = np.random.normal(0, 0.001, image.shape)
    noisy_image = image + noise
    aug_images.append(np.clip(noisy_image, 0, 1))  # Ensure pixel values remain in [0, 1]

    # Mild rotation (reduce the angle range)
    rows, cols = image.shape[:2]
    angle = random.choice([-5, 5])  # Smaller angles to avoid heavy distortion
    M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
    rotated_image = cv2.warpAffine(image, M, (cols, rows), borderMode=cv2.BORDER_REFLECT)
    aug_images.append(rotated_image)

    return aug_images

# Loop through all subfolders and images
for subfolder in os.listdir(data_dir):
    subfolder_path = os.path.join(data_dir, subfolder)
    
    # Ensure the path is a directory
    if os.path.isdir(subfolder_path):
        # Create corresponding subfolder in the save directory
        save_subfolder = os.path.join(save_dir, subfolder)
        if not os.path.exists(save_subfolder):
            os.makedirs(save_subfolder)
        
        # Loop through all images in the current subfolder
        for img_name in tqdm(os.listdir(subfolder_path)):
            # Read the image
            img_path = os.path.join(subfolder_path, img_name)
            image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Load as grayscale
            if image is None:
                continue

            # Step 1: Resize to 256x256 (preserving aspect ratio if needed)
            resized_image = cv2.resize(image, (256, 256))

            # Step 2: Normalize (Z-score normalization)
            normalized_image = normalize_image(resized_image)

            # Step 3: Apply conservative augmentation
            augmented_images = augment_image(normalized_image)

            # Save the processed images
            base_name, ext = os.path.splitext(img_name)
            cv2.imwrite(os.path.join(save_subfolder, f"{base_name}_resized{ext}"), (normalized_image * 255).astype(np.uint8))

            # Save augmented images (keeping it minimal)
            for i, aug_image in enumerate(augmented_images):
                cv2.imwrite(os.path.join(save_subfolder, f"{base_name}_aug_{i}{ext}"), (aug_image * 255).astype(np.uint8))

print(f"Preprocessing complete. Processed images saved to {save_dir}.")
